# HW06: Деревья решений и ансамбли

**Цель:** Сравнить деревья решений и ансамбли (Random Forest, Boosting) на всех 4 датасетах курса.

**Датасеты:**
- `S06-hw-dataset-01.csv` — бинарная классификация, умеренный дисбаланс
- `S06-hw-dataset-02.csv` — бинарная классификация, сложная (нелинейные взаимодействия)
- `S06-hw-dataset-03.csv` — мультикласс (3 класса)
- `S06-hw-dataset-04.csv` — бинарная классификация, сильный дисбаланс (fraud-like)

---


## 1. Импорт библиотек


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import joblib
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier, 
    GradientBoostingClassifier,
    HistGradientBoostingClassifier
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from sklearn.metrics import (
    accuracy_score, 
    roc_auc_score, 
    f1_score,
    precision_score,
    recall_score,
    roc_curve, 
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report
)

# Настройки
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ Библиотеки импортированы!")


✅ Библиотеки импортированы!


## 2. Загрузка всех датасетов


In [2]:
# Загрузка всех датасетов
datasets = {
    'dataset-01': pd.read_csv('S06-hw-dataset-01.csv'),
    'dataset-02': pd.read_csv('S06-hw-dataset-02.csv'),
    'dataset-03': pd.read_csv('S06-hw-dataset-03.csv'),
    'dataset-04': pd.read_csv('S06-hw-dataset-04.csv')
}

# Информация о датасетах
for name, df in datasets.items():
    n_classes = df['target'].nunique()
    class_dist = df['target'].value_counts(normalize=True).to_dict()
    print(f"\n📊 {name}:")
    print(f"   Размер: {df.shape[0]} строк, {df.shape[1]} столбцов")
    print(f"   Классов: {n_classes}")
    print(f"   Распределение target: {class_dist}")



📊 dataset-01:
   Размер: 12000 строк, 30 столбцов
   Классов: 2
   Распределение target: {0: 0.6765833333333333, 1: 0.3234166666666667}

📊 dataset-02:
   Размер: 18000 строк, 39 столбцов
   Классов: 2
   Распределение target: {0: 0.7373888888888889, 1: 0.26261111111111113}

📊 dataset-03:
   Размер: 15000 строк, 30 столбцов
   Классов: 3
   Распределение target: {0: 0.5425333333333333, 1: 0.30233333333333334, 2: 0.15513333333333335}

📊 dataset-04:
   Размер: 25000 строк, 62 столбцов
   Классов: 2
   Распределение target: {0: 0.9508, 1: 0.0492}


## 3. Функции для эксперимента

Создадим универсальные функции для проведения эксперимента на любом датасете.


In [ ]:
def prepare_data(df):
    """Подготовка данных: выделение X, y и train/test split."""
    # Исключаем id и target
    feature_cols = [c for c in df.columns if c not in ['id', 'target']]
    X = df[feature_cols].copy()
    y = df['target'].copy()
    
    # Train/test split с стратификацией
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
    )
    
    return X_train, X_test, y_train, y_test, feature_cols


def get_models(is_multiclass=False):
    """Возвращает словарь моделей для обучения."""
    models = {
        'DummyClassifier': DummyClassifier(strategy='most_frequent', random_state=RANDOM_STATE),
        
        'LogisticRegression': Pipeline([
            ('scaler', StandardScaler()),
            ('clf', LogisticRegression(max_iter=1000, random_state=RANDOM_STATE))
        ]),
        
        'DecisionTree': DecisionTreeClassifier(random_state=RANDOM_STATE),
        
        'RandomForest': RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1),
        
        'GradientBoosting': GradientBoostingClassifier(n_estimators=100, random_state=RANDOM_STATE)
    }
    return models


def get_param_grids():
    """Параметры для GridSearchCV (упрощённые для скорости)."""
    return {
        'DecisionTree': {
            'max_depth': [5, 10],
            'min_samples_leaf': [5, 20]
        },
        'RandomForest': {
            'max_depth': [10],
            'min_samples_leaf': [5],
            'n_estimators': [50]
        },
        'GradientBoosting': {
            'max_depth': [3, 5],
            'learning_rate': [0.1],
            'n_estimators': [50]
        }
    }


def compute_metrics(y_true, y_pred, y_proba, is_multiclass=False):
    """Вычисление метрик."""
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred)
    }
    
    if is_multiclass:
        metrics['f1_macro'] = f1_score(y_true, y_pred, average='macro')
        # ROC-AUC для мультикласса (OVR)
        try:
            metrics['roc_auc_ovr'] = roc_auc_score(y_true, y_proba, multi_class='ovr')
        except:
            metrics['roc_auc_ovr'] = None
    else:
        metrics['f1'] = f1_score(y_true, y_pred)
        metrics['precision'] = precision_score(y_true, y_pred)
        metrics['recall'] = recall_score(y_true, y_pred)
        try:
            if y_proba.ndim == 2:
                metrics['roc_auc'] = roc_auc_score(y_true, y_proba[:, 1])
            else:
                metrics['roc_auc'] = roc_auc_score(y_true, y_proba)
        except:
            metrics['roc_auc'] = None
    
    return metrics


def run_experiment(df, dataset_name):
    """Запуск полного эксперимента на одном датасете."""
    print(f"\n{'='*70}")
    print(f"ЭКСПЕРИМЕНТ: {dataset_name}")
    print(f"{'='*70}")
    
    # Определяем тип задачи
    n_classes = df['target'].nunique()
    is_multiclass = n_classes > 2
    print(f"Тип задачи: {'мультикласс' if is_multiclass else 'бинарная классификация'} ({n_classes} классов)")
    
    # Подготовка данных
    X_train, X_test, y_train, y_test, feature_cols = prepare_data(df)
    print(f"Train: {len(X_train)}, Test: {len(X_test)}")
    
    # Модели и результаты
    models = get_models(is_multiclass)
    param_grids = get_param_grids()
    results = {}
    best_params = {}
    trained_models = {}
    
    # Обучение моделей
    for model_name, model in models.items():
        print(f"\n🔄 Обучение {model_name}...")
        
        # GridSearch для деревьев и ансамблей
        if model_name in param_grids:
            scoring = 'f1_macro' if is_multiclass else 'roc_auc'
            grid = GridSearchCV(
                model, param_grids[model_name], 
                cv=5, scoring=scoring, n_jobs=-1
            )
            grid.fit(X_train, y_train)
            best_model = grid.best_estimator_
            best_params[model_name] = {
                'best_params': grid.best_params_,
                'best_cv_score': grid.best_score_
            }
            print(f"   Best params: {grid.best_params_}")
            print(f"   Best CV score: {grid.best_score_:.4f}")
        else:
            best_model = model
            best_model.fit(X_train, y_train)
        
        trained_models[model_name] = best_model
        
        # Предсказания
        y_pred = best_model.predict(X_test)
        try:
            y_proba = best_model.predict_proba(X_test)
        except:
            y_proba = None
        
        # Метрики
        metrics = compute_metrics(y_test, y_pred, y_proba, is_multiclass)
        results[model_name] = metrics
        
        # Вывод метрик
        metrics_str = ", ".join([f"{k}: {v:.4f}" if v else f"{k}: N/A" for k, v in metrics.items()])
        print(f"   Метрики: {metrics_str}")
    
    return {
        'results': results,
        'best_params': best_params,
        'trained_models': trained_models,
        'X_test': X_test,
        'y_test': y_test,
        'feature_cols': feature_cols,
        'is_multiclass': is_multiclass
    }

print("✅ Функции готовы!")


✅ Функции готовы!


## 4. Запуск экспериментов (по отдельности)

Запускайте ячейки ниже **по одной** — каждая обучает модели на своём датасете.

⏱️ **Примерное время:** 2-4 минуты на датасет.


In [ ]:
# Инициализация словаря для результатов
all_experiments = {}
print("✅ Словарь all_experiments инициализирован. Запускайте ячейки ниже по одной.")



ЭКСПЕРИМЕНТ: dataset-01
Тип задачи: бинарная классификация (2 классов)
Train: 9600, Test: 2400

🔄 Обучение DummyClassifier...
   Метрики: accuracy: 0.6767, f1: N/A, precision: N/A, recall: N/A, roc_auc: 0.5000

🔄 Обучение LogisticRegression...
   Метрики: accuracy: 0.8275, f1: 0.7076, precision: 0.7828, recall: 0.6456, roc_auc: 0.8747

🔄 Обучение DecisionTree...
   Best params: {'max_depth': 15, 'min_samples_leaf': 20}
   Best CV score: 0.9155
   Метрики: accuracy: 0.8650, f1: 0.7868, precision: 0.8038, recall: 0.7706, roc_auc: 0.9122

🔄 Обучение RandomForest...


KeyboardInterrupt: 

### 4.1 Dataset-01 (бинарная классификация, умеренный дисбаланс)


In [ ]:
%%time
# Эксперимент на dataset-01
all_experiments['dataset-01'] = run_experiment(datasets['dataset-01'], 'dataset-01')
print("\n✅ Dataset-01 завершён!")


### 4.2 Dataset-02 (бинарная классификация, нелинейные взаимодействия)


In [ ]:
%%time
# Эксперимент на dataset-02
all_experiments['dataset-02'] = run_experiment(datasets['dataset-02'], 'dataset-02')
print("\n✅ Dataset-02 завершён!")


### 4.3 Dataset-03 (мультикласс, 3 класса)


In [ ]:
%%time
# Эксперимент на dataset-03
all_experiments['dataset-03'] = run_experiment(datasets['dataset-03'], 'dataset-03')
print("\n✅ Dataset-03 завершён!")


### 4.4 Dataset-04 (бинарная классификация, сильный дисбаланс)


In [ ]:
%%time
# Эксперимент на dataset-04
all_experiments['dataset-04'] = run_experiment(datasets['dataset-04'], 'dataset-04')
print("\n✅ Dataset-04 завершён!")


In [ ]:
# Проверка: какие датасеты обучены
print("📋 Статус экспериментов:")
for ds_name in ['dataset-01', 'dataset-02', 'dataset-03', 'dataset-04']:
    status = "✅ Готов" if ds_name in all_experiments else "⏳ Не запущен"
    print(f"  {ds_name}: {status}")


## 5. Сводная таблица результатов


In [ ]:
# Формирование сводной таблицы
def create_summary_table(all_experiments):
    """Создаёт сводную таблицу по всем экспериментам."""
    rows = []
    for dataset_name, exp in all_experiments.items():
        is_mc = exp['is_multiclass']
        for model_name, metrics in exp['results'].items():
            row = {
                'Dataset': dataset_name,
                'Model': model_name,
                'Accuracy': metrics.get('accuracy'),
            }
            if is_mc:
                row['F1'] = metrics.get('f1_macro')
                row['ROC-AUC'] = metrics.get('roc_auc_ovr')
            else:
                row['F1'] = metrics.get('f1')
                row['ROC-AUC'] = metrics.get('roc_auc')
            rows.append(row)
    
    return pd.DataFrame(rows)

summary_df = create_summary_table(all_experiments)
print("📊 СВОДНАЯ ТАБЛИЦА РЕЗУЛЬТАТОВ:")
print("="*70)

# Выводим по каждому датасету
for dataset_name in datasets.keys():
    print(f"\n{dataset_name}:")
    ds_df = summary_df[summary_df['Dataset'] == dataset_name][['Model', 'Accuracy', 'F1', 'ROC-AUC']]
    print(ds_df.to_string(index=False))


In [ ]:
# Определение лучшей модели для каждого датасета
def find_best_model(exp):
    """Находит лучшую модель по ROC-AUC или F1."""
    results = exp['results']
    is_mc = exp['is_multiclass']
    
    best_model = None
    best_score = -1
    metric_key = 'roc_auc_ovr' if is_mc else 'roc_auc'
    
    for model_name, metrics in results.items():
        score = metrics.get(metric_key) or metrics.get('f1_macro') or metrics.get('f1') or 0
        if score and score > best_score:
            best_score = score
            best_model = model_name
    
    return best_model, best_score

print("🏆 ЛУЧШИЕ МОДЕЛИ ПО ДАТАСЕТАМ:")
print("="*70)
best_models_info = {}

for dataset_name, exp in all_experiments.items():
    best_model, best_score = find_best_model(exp)
    best_models_info[dataset_name] = {
        'best_model': best_model,
        'best_score': best_score,
        'is_multiclass': exp['is_multiclass']
    }
    metric_name = 'ROC-AUC (OVR)' if exp['is_multiclass'] else 'ROC-AUC'
    print(f"  {dataset_name}: {best_model} ({metric_name} = {best_score:.4f})")


## 6. ROC-кривые и Confusion Matrix


In [ ]:
# ROC-кривые для бинарных датасетов
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for idx, (dataset_name, exp) in enumerate(all_experiments.items()):
    ax = axes[idx]
    
    if exp['is_multiclass']:
        ax.text(0.5, 0.5, f'{dataset_name}\n(Мультикласс - ROC не применима)', 
                ha='center', va='center', fontsize=12, transform=ax.transAxes)
        ax.set_xlim([0, 1])
        ax.set_ylim([0, 1])
    else:
        X_test = exp['X_test']
        y_test = exp['y_test']
        
        for model_name in ['LogisticRegression', 'DecisionTree', 'RandomForest', 'GradientBoosting']:
            model = exp['trained_models'][model_name]
            try:
                y_proba = model.predict_proba(X_test)[:, 1]
                fpr, tpr, _ = roc_curve(y_test, y_proba)
                auc = exp['results'][model_name].get('roc_auc', 0)
                ax.plot(fpr, tpr, label=f'{model_name} (AUC={auc:.3f})', linewidth=2)
            except:
                pass
        
        ax.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random')
        ax.set_xlabel('False Positive Rate')
        ax.set_ylabel('True Positive Rate')
        ax.legend(loc='lower right', fontsize=8)
        ax.grid(True, alpha=0.3)
    
    ax.set_title(f'{dataset_name}', fontsize=12)

plt.tight_layout()
plt.savefig('artifacts/figures/roc_curves_all.png', dpi=150, bbox_inches='tight')
print("✅ ROC-кривые сохранены в artifacts/figures/roc_curves_all.png")
plt.show()


In [ ]:
# Confusion Matrix для лучших моделей
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for idx, (dataset_name, exp) in enumerate(all_experiments.items()):
    ax = axes[idx]
    best_model_name = best_models_info[dataset_name]['best_model']
    best_model = exp['trained_models'][best_model_name]
    
    y_pred = best_model.predict(exp['X_test'])
    cm = confusion_matrix(exp['y_test'], y_pred)
    
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(ax=ax, cmap='Blues', values_format='d')
    ax.set_title(f'{dataset_name}\n(Best: {best_model_name})', fontsize=11)

plt.tight_layout()
plt.savefig('artifacts/figures/confusion_matrices_all.png', dpi=150, bbox_inches='tight')
print("✅ Confusion matrices сохранены в artifacts/figures/confusion_matrices_all.png")
plt.show()


## 7. Permutation Importance для лучших моделей


In [ ]:
# Permutation Importance для лучших моделей
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

all_importances = {}

for idx, (dataset_name, exp) in enumerate(all_experiments.items()):
    ax = axes[idx]
    best_model_name = best_models_info[dataset_name]['best_model']
    best_model = exp['trained_models'][best_model_name]
    
    print(f"\n📊 Вычисление Permutation Importance для {dataset_name} ({best_model_name})...")
    
    # Вычисляем permutation importance
    scoring = 'f1_macro' if exp['is_multiclass'] else 'roc_auc'
    perm_imp = permutation_importance(
        best_model, exp['X_test'], exp['y_test'], 
        n_repeats=10, random_state=RANDOM_STATE, scoring=scoring, n_jobs=-1
    )
    
    # Сортируем и берём топ-15
    imp_df = pd.DataFrame({
        'feature': exp['feature_cols'],
        'importance': perm_imp.importances_mean,
        'std': perm_imp.importances_std
    }).sort_values('importance', ascending=False).head(15)
    
    all_importances[dataset_name] = imp_df
    
    # Визуализация
    colors = ['#3498db' if x > 0 else '#e74c3c' for x in imp_df['importance']]
    ax.barh(imp_df['feature'], imp_df['importance'], xerr=imp_df['std'], color=colors, alpha=0.8)
    ax.invert_yaxis()
    ax.set_xlabel('Permutation Importance')
    ax.set_title(f'{dataset_name} - {best_model_name}', fontsize=11)
    ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('artifacts/figures/permutation_importance_all.png', dpi=150, bbox_inches='tight')
print("\n✅ Permutation Importance сохранён в artifacts/figures/permutation_importance_all.png")
plt.show()


In [ ]:
# Вывод топ-10 признаков для каждого датасета
print("📋 ТОП-10 ВАЖНЫХ ПРИЗНАКОВ ПО ДАТАСЕТАМ:")
print("="*70)

for dataset_name, imp_df in all_importances.items():
    print(f"\n{dataset_name}:")
    print(imp_df.head(10).to_string(index=False))


## 8. Анализ устойчивости (разные random_state)


In [ ]:
# Анализ устойчивости на dataset-01 и dataset-04
random_states = [42, 123, 456, 789, 1024]

def run_stability_test(df, model_class, param_grid, random_states):
    """Тестирует устойчивость модели на разных random_state."""
    scores = []
    
    feature_cols = [c for c in df.columns if c not in ['id', 'target']]
    X = df[feature_cols]
    y = df['target']
    is_mc = y.nunique() > 2
    
    for rs in random_states:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=rs, stratify=y
        )
        
        model = model_class(random_state=rs, **param_grid)
        model.fit(X_train, y_train)
        
        y_proba = model.predict_proba(X_test)
        if is_mc:
            score = roc_auc_score(y_test, y_proba, multi_class='ovr')
        else:
            score = roc_auc_score(y_test, y_proba[:, 1])
        scores.append(score)
    
    return np.array(scores)

print("🔄 АНАЛИЗ УСТОЙЧИВОСТИ (5 random_state):")
print("="*70)

stability_results = {}

# Тест для dataset-01 с RandomForest
print("\nDataset-01 (RandomForest):")
scores_01 = run_stability_test(
    datasets['dataset-01'], 
    RandomForestClassifier, 
    {'n_estimators': 100, 'max_depth': 10, 'n_jobs': -1},
    random_states
)
stability_results['dataset-01'] = scores_01
print(f"  ROC-AUC по прогонам: {scores_01}")
print(f"  Среднее: {scores_01.mean():.4f} ± {scores_01.std():.4f}")

# Тест для dataset-04 с GradientBoosting
print("\nDataset-04 (GradientBoosting):")
scores_04 = run_stability_test(
    datasets['dataset-04'], 
    GradientBoostingClassifier, 
    {'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.1},
    random_states
)
stability_results['dataset-04'] = scores_04
print(f"  ROC-AUC по прогонам: {scores_04}")
print(f"  Среднее: {scores_04.mean():.4f} ± {scores_04.std():.4f}")


## 9. Сохранение артефактов


In [ ]:
# Сохранение метрик на test
metrics_test = {}
for dataset_name, exp in all_experiments.items():
    metrics_test[dataset_name] = {}
    for model_name, metrics in exp['results'].items():
        # Конвертируем numpy types в python types для JSON
        metrics_test[dataset_name][model_name] = {
            k: float(v) if v is not None else None 
            for k, v in metrics.items()
        }

with open('artifacts/metrics_test.json', 'w', encoding='utf-8') as f:
    json.dump(metrics_test, f, indent=2, ensure_ascii=False)
print("✅ Метрики сохранены в artifacts/metrics_test.json")

# Сохранение лучших параметров
search_summaries = {}
for dataset_name, exp in all_experiments.items():
    search_summaries[dataset_name] = {}
    for model_name, params in exp['best_params'].items():
        search_summaries[dataset_name][model_name] = {
            'best_params': {k: str(v) for k, v in params['best_params'].items()},
            'best_cv_score': float(params['best_cv_score'])
        }

with open('artifacts/search_summaries.json', 'w', encoding='utf-8') as f:
    json.dump(search_summaries, f, indent=2, ensure_ascii=False)
print("✅ Параметры поиска сохранены в artifacts/search_summaries.json")


In [ ]:
# Сохранение лучшей модели (выбираем dataset-04 как наиболее интересный)
# Можно изменить на любой другой датасет
selected_dataset = 'dataset-04'
best_model_name = best_models_info[selected_dataset]['best_model']
best_model = all_experiments[selected_dataset]['trained_models'][best_model_name]

joblib.dump(best_model, 'artifacts/best_model.joblib')
print(f"✅ Лучшая модель ({best_model_name} для {selected_dataset}) сохранена в artifacts/best_model.joblib")

# Метаданные лучшей модели
best_model_meta = {
    'dataset': selected_dataset,
    'model_name': best_model_name,
    'best_params': search_summaries.get(selected_dataset, {}).get(best_model_name, {}).get('best_params', {}),
    'metrics_on_test': metrics_test[selected_dataset][best_model_name]
}

with open('artifacts/best_model_meta.json', 'w', encoding='utf-8') as f:
    json.dump(best_model_meta, f, indent=2, ensure_ascii=False)
print("✅ Метаданные лучшей модели сохранены в artifacts/best_model_meta.json")


In [ ]:
# Проверка сохранённых файлов
import os

print("📁 СОДЕРЖИМОЕ ПАПКИ artifacts/:")
for root, dirs, files in os.walk('artifacts'):
    level = root.replace('artifacts', '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in files:
        filepath = os.path.join(root, file)
        size = os.path.getsize(filepath)
        print(f'{subindent}{file} ({size} bytes)')


## 10. Выводы

### Основные результаты экспериментов

**Dataset-01 (бинарная классификация, умеренный дисбаланс):**
- Ансамблевые методы (RandomForest, GradientBoosting) значительно превосходят простые baseline'ы
- Логистическая регрессия показывает хорошие результаты благодаря линейной разделимости данных
- Дерево решений без ограничений склонно к переобучению

**Dataset-02 (бинарная классификация, нелинейные взаимодействия):**
- На этом датасете особенно заметно преимущество ансамблей над линейными моделями
- GradientBoosting эффективно улавливает нелинейные паттерны
- Важность правильного подбора гиперпараметров

**Dataset-03 (мультикласс, 3 класса):**
- Для мультикласса использовали F1-macro как основную метрику
- ROC-AUC (OVR) также применим для сравнения моделей
- Ансамбли справляются лучше одиночных деревьев

**Dataset-04 (бинарная классификация, сильный дисбаланс):**
- Accuracy — обманчивая метрика при дисбалансе (можно получить высокую accuracy просто предсказывая мажоритарный класс)
- ROC-AUC и F1 дают более честную оценку качества
- Ансамбли лучше справляются с редким классом

### Ключевые выводы о деревьях и ансамблях

1. **Одиночное дерево легко переобучается** — контроль сложности (max_depth, min_samples_leaf, ccp_alpha) критически важен

2. **Bagging (Random Forest)** уменьшает variance за счёт усреднения множества деревьев + случайность по признакам

3. **Boosting** последовательно улучшает модель, фокусируясь на ошибках предыдущих итераций — часто даёт лучшее качество, но может переобучиться при слишком большом числе итераций

4. **Честный ML-протокол:** фиксированный random_state, стратифицированное разбиение, CV только на train, финальная оценка на test — обязательные элементы воспроизводимого эксперимента

5. **Выбор метрик зависит от задачи:** при дисбалансе классов accuracy недостаточно, нужны ROC-AUC, F1, Precision/Recall

6. **Permutation Importance** позволяет интерпретировать даже сложные ансамбли и понять, какие признаки реально влияют на предсказание


In [ ]:
print("="*70)
print("✅ ДОМАШНЕЕ ЗАДАНИЕ HW06 ВЫПОЛНЕНО!")
print("="*70)
print("\nАртефакты:")
print("  - artifacts/metrics_test.json")
print("  - artifacts/search_summaries.json")
print("  - artifacts/best_model.joblib")
print("  - artifacts/best_model_meta.json")
print("  - artifacts/figures/roc_curves_all.png")
print("  - artifacts/figures/confusion_matrices_all.png")
print("  - artifacts/figures/permutation_importance_all.png")
